In [1]:
import time
import pandas as pd
import re # regular expressions like * & $

from selenium.webdriver import Chrome, Firefox
from selenium.webdriver.common.keys import Keys # use in performing action of keyboard event
from selenium.webdriver.common.by import By # use in locating elements  
from selenium.common.exceptions import NoSuchElementException # post requisite in proofing there are no more elements exist

# creating scroll down function to handle lazy loading problem
def scrollDown (driver, n_scroll):
    body = driver.find_element_by_tag_name("body")
    while n_scroll >= 0:
        body.send_keys(Keys.PAGE_DOWN)
        n_scroll -= 1
    return driver

# open firefox browser and navigate to the page
driver = Firefox(executable_path='C:/Program Files/Mozilla Firefox/geckodriver.exe')
driver.get("https://www.sephora.my")

# catagorize each product which will be used to scrap
productcat = ['cleanser-and-exfoliator', 'toner', 'moisturiser', 'masks-and-treatments', 'suncare']

In [2]:
# create dataframe
df = pd.DataFrame(columns=['Category', 'URL'])
print(df)

Empty DataFrame
Columns: [Category, URL]
Index: []


In [ ]:
# navigate to link and search categories that we have categorize earlier
for cat in productcat:
    driver.get('https://www.sephora.my/categories/skincare/' + cat + '?view=120') 
    time.sleep(1) # time delay
    
    browser = scrollDown(driver, 10)
    time.sleep(1)
    
    browser = scrollDown(driver, 10)
    time.sleep(1)
    
    browser = scrollDown(driver, 10)
    time.sleep(1)
    
    browser = scrollDown(driver, 10)
    
    # find element with class name (go through one by one image of skincare products)
    element = driver.find_elements_by_class_name('product-card-image-container')
    
    # any products that contains a link (which can be redirected to) will be stored in an array  
    productURL = []
    for a in element:
        subURL = a.get_attribute('href')
        productURL.append(subURL)
    
    # transform all these link array data into dataframe
    dic = {'Category': cat, 'URL': productURL}
    df = df.append(pd.DataFrame(dic), ignore_index = True)

In [4]:
# add more columns with more categories into dataframe
df_info = pd.DataFrame(columns=['brand', 'name', 'price', 'skin_type', 'skin_concern', 'rating'])
df = pd.concat([df, df_info], axis = 1)

In [5]:
# step 2 (scrape with different categories)

# within the respective url of dataframe scrape each category
for i in range(len(df)):
    driver.get(df.URL[i])
    time.sleep(5)
    
    # find any tags with these class name and scrape them
    
    # brand, name, price
    try:
        df.brand[i] = driver.find_element_by_class_name('product-brand').text
        df.name[i] = driver.find_element_by_class_name('product-heading').text
        df.price[i] = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/main/div/div/div[2]/div[2]/div[1]/div[2]/div[1]/p/span').text
    
    except:
        pass
    
    # skin_type
    try:
        df.skin_type[i] = driver.find_element_by_class_name('product-filter-type-value').text
    
    except:
        pass
    
    # skin concern
    try:
        df.skin_concern[i] = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/main/div/div/div[2]/div[2]/div[2]/div[1]/div/div[1]/p[2]/span[2]').text
    
    except NoSuchElementException:
        pass

    # rating
    try:
        rating = driver.find_element_by_class_name('rateit-range').get_attribute('aria-valuenow')
        df['rating'][i] = rating
        
    except NoSuchElementException:
        df['rating'][i] = 0
        
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


InvalidSessionIdException: Message: Tried to run command without establishing a connection


In [7]:
# convert dataframe to csv file
df.to_csv('sephoramy_webscrape_dataset_raw.csv', encoding = 'utf-8-sig', index = False)

In [6]:
df.head(10)

,Category,URL,brand,name,price,skin_type,skin_concern,rating
0,cleanser-and-exfoliator,https://www.sephora.my/products/fresh-soy-face...,FRESH,NaN,NaN,NaN,NaN,4.5
1,cleanser-and-exfoliator,https://www.sephora.my/products/the-inkey-list...,THE INKEY LIST,150ml,RM46.00,NaN,NaN,4
2,cleanser-and-exfoliator,https://www.sephora.my/products/fresh-travel-e...,FRESH,NaN,NaN,NaN,NaN,4.5
3,cleanser-and-exfoliator,https://www.sephora.my/products/sephora-collec...,SEPHORA COLLECTION,70ml,RM27.00,NaN,NaN,4
4,cleanser-and-exfoliator,https://www.sephora.my/products/fenty-skin-sta...,FENTY SKIN,NaN,NaN,NaN,NaN,5
5,cleanser-and-exfoliator,https://www.sephora.my/products/dr-dennis-gros...,DR. DENNIS GROSS,NaN,NaN,NaN,NaN,4.5
6,cleanser-and-exfoliator,https://www.sephora.my/products/bobbi-brown-sw...,BOBBI BROWN,NaN,NaN,NaN,NaN,4
7,cleanser-and-exfoliator,https://www.sephora.my/products/sk-ii-facial-t...,SK-II,120g,RM249.00,NaN,NaN,4.5
8,cleanser-and-exfoliator,https://www.sephora.my/products/sephora-collec...,SEPHORA COLLECTION,NaN,NaN,NaN,NaN,4
9,cleanser-and-exfoliator,https://www.sephora.my/products/dr-dot-jart-pl...,DR.JART+,NaN,NaN,NaN,NaN,4.5
